# Day 11: Cosmic Expansion

## Part 1

You continue following signs for "Hot Springs" and eventually come across an observatory. The Elf within turns out to be a researcher studying cosmic expansion using the giant telescope here.

He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight there once he's done with today's observation analysis.

Maybe you can help him with the analysis to speed things up?

The researcher has collected a bunch of data and compiled the data into a single giant image (your puzzle input). The image includes empty space (`.`) and galaxies (`#`). For example:

```
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
```

The researcher is trying to figure out the sum of the lengths of the shortest path between every pair of galaxies. However, there's a catch: the universe expanded in the time it took the light from those galaxies to reach the observatory.

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.

In the above example, three columns and two rows contain no galaxies:

```
   v  v  v
 ...#......
 .......#..
 #.........
>..........<
 ......#...
 .#........
 .........#
>..........<
 .......#..
 #...#.....
   ^  ^  ^
```

These rows and columns need to be twice as big; the result of cosmic expansion therefore looks like this:

```
....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
```

Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help to assign every galaxy a unique number:

```
....1........
.........2...
3............
.............
.............
........4....
.5...........
............6
.............
.............
.........7...
8....9.......
```

In these 9 galaxies, there are 36 pairs. Only count each pair once; order within the pair doesn't matter. For each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right exactly one `.` or `#` at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)

For example, here is one of the shortest paths between galaxies `5` and `9`:

```
....1........
.........2...
3............
.............
.............
........4....
.5...........
.##.........6
..##.........
...##........
....##...7...
8....9.......
```

This path has length `9` because it takes a minimum of nine steps to get from galaxy `5` to galaxy `9` (the eight locations marked # plus the step onto galaxy 9 itself). Here are some other example shortest path lengths:

- Between galaxy `1` and galaxy `7`: `15`
- Between galaxy `3` and galaxy `6`: `17`
- Between galaxy `8` and galaxy `9`: `5`

In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is `374`.

Expand the universe, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

In [49]:
import numpy as np


test_data_raw = (
"""...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""
)


def parse_data(data):
    return np.array([list(line) for line in data.splitlines()])


def print_data(data):
    for line in data:
        print(''.join([str(s) for s in line]))


test_data = parse_data(test_data_raw)
print_data(test_data)

...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....


In [88]:
def expand(data):
    result = np.copy(data)
    for axis in [0, 1]:
        indices = []
        for idx in range(data.shape[axis]):
            indices.append(idx)
            if np.all(np.take(data, idx, axis=axis) == "."):
                indices.append(idx)
        result = np.take(result, indices, axis=axis)
    return result


expected_test_data_expanded = parse_data(
"""....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#......."""
)

test_data_expanded = expand(test_data)
print("EXPANDED", test_data_expanded.shape)
print_data(test_data_expanded)
print("EXPECTED", expected_test_data_expanded.shape)
print_data(expected_test_data_expanded)

EXPANDED (12, 13)
....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
EXPECTED (12, 13)
....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......


In [59]:
def get_galaxies_loc(data):
    xs, ys = np.where(data == "#")
    return list(zip(xs, ys))


def get_galaxies_pairs(data):
    galaxies = get_galaxies_loc(data)
    for idx1, point1 in enumerate(galaxies):
        for idx2, point2 in enumerate(galaxies[idx1 + 1:]):
            yield idx1 + 1, point1, idx1 + idx2 + 2 ,point2


def get_shortests_paths(data):
    result = {}
    for id1, point1, id2, point2 in get_galaxies_pairs(data):
        result[(id1, id2)] = abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])
    return result


get_shortests_paths(expand(test_data))

{(1, 2): 6,
 (1, 3): 6,
 (1, 4): 9,
 (1, 5): 9,
 (1, 6): 15,
 (1, 7): 15,
 (1, 8): 15,
 (1, 9): 12,
 (2, 3): 10,
 (2, 4): 5,
 (2, 5): 13,
 (2, 6): 9,
 (2, 7): 9,
 (2, 8): 19,
 (2, 9): 14,
 (3, 4): 11,
 (3, 5): 5,
 (3, 6): 17,
 (3, 7): 17,
 (3, 8): 9,
 (3, 9): 14,
 (4, 5): 8,
 (4, 6): 6,
 (4, 7): 6,
 (4, 8): 14,
 (4, 9): 9,
 (5, 6): 12,
 (5, 7): 12,
 (5, 8): 6,
 (5, 9): 9,
 (6, 7): 6,
 (6, 8): 16,
 (6, 9): 11,
 (7, 8): 10,
 (7, 9): 5,
 (8, 9): 5}

In [60]:
def part1(data):
    data = expand(data)
    paths = get_shortests_paths(data)
    return sum(paths.values())


part1(test_data)

374

In [61]:
with open("input.txt") as f:
    data = parse_data(f.read())

print_data(data[:10, :10])

..........
..........
..........
...#......
..........
..........
..........
..........
..........
..........


In [62]:
part1(data)

9522407

## Part 2

The galaxies are much older (and thus much farther apart) than the researcher initially estimated.

Now, instead of the expansion you did before, make each empty row or column one million times larger. That is, each empty row should be replaced with `1000000` empty rows, and each empty column should be replaced with `1000000` empty columns.

(In the example above, if each empty row or column were merely `10` times larger, the sum of the shortest paths between every pair of galaxies would be `1030`. If each empty row or column were merely `100` times larger, the sum of the shortest paths between every pair of galaxies would be `8410`. However, your universe will need to expand far beyond these values.)

Starting with the same initial image, expand the universe according to these new rules, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

In [118]:
def part2(data, empty_distance_base=1000000):
    empty = [
        np.array([idx for idx in range(data.shape[axis]) if np.all(np.take(data, idx, axis=axis) == ".")])
        for axis in [0, 1]
    ]
    distances = {}

    for id1, point1, id2, point2 in get_galaxies_pairs(data):
        empty_distance = sum([
            np.sum((empty[axis] > min(point1[axis], point2[axis])) & (empty[axis] < max(point1[axis], point2[axis])))
            for axis in [0, 1]
        ])
        distance = abs(point1[0] - point2[0]) + empty_distance * (empty_distance_base - 1) + abs(point1[1] - point2[1])
        distances[(id1, id2)] = distance

    return sum(distances.values())


print(part2(expand(test_data), 1))
print(part2(test_data, 10))
print(part2(test_data, 100))

374
1030
8410


In [98]:
part2(data)

544723432977